# Azure ML Pipelines

Цель лабораторной работы: 

- создание с __Конвейеров машинного обучения__ (ML Pipelines) в Azure ML
- тренировка модели машинного обучения.

## Подготовка 

Импорт необходимых модулей и проверка версии AzureML SDK:

In [12]:
import azureml.core
from azureml.core import Workspace, Environment, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

SDK version: 1.14.0


Получим конфигурацию эксперимента: 

In [6]:
%run core.py

config = get_experiment_config('lab_6A')
init_experiment(config)
experiment_dir = get_experiment_dir(config)

config

Experiment pipeline-experiment was initialized successfully.


{'experiment_name': 'pipeline-experiment',
 'working_subdir': 'pipeline-experiment-lab',
 'environment_name': 'environments-experiment',
 'core': {'expriments_root_dir': 'experiments/',
  'datastore_name': 'aml_ws_datastore_v2',
  'dataset_name': 'diabetes-data',
  'ml_cluster_name': 'aml-ws-cluster',
  'ml_model_name': 'diabetes-predict-model'}}

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [4]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

Successfully connected to Workspace: aml-workshop.


## Зададим Шаги ML Конвейера

Зададим и сохраним скрипт для Шага регистрации модели:

In [5]:
%%writefile scripts/register_model.py

#%% Import libraries
import argparse
import joblib
from azureml.core import Workspace, Model, Run


#%% Get parameters
parser = argparse.ArgumentParser()
parser.add_argument('--model_dir', type=str, dest='model_dir', default='outputs', help='Define ML model directory')
args = parser.parse_args()


#%% Set model directory
model_dir = args.model_dir


#%% Get the experiment run context
run = Run.get_context()


#%% Load the model
model_path = f'{model_dir}/model.pkl'

print(f'Loading {model_path}...')
model = joblib.load(model_path)


#%% Register model
Model.register(workspace=run.experiment.workspace,
               model_path=model_path,
               model_name='diabetes-predict-model',
               tags={'Lab': '5A'})

#%% Finish Experiment
run.complete()

Overwriting scripts/register_model.py


## Создадим и запустим ML Конвейер

Подготовка к созданию:

In [7]:
# Get dataset
data_ds = ws.datasets.get(config['core']['dataset_name'])
print(f'Used dataset {data_ds.name}: {data_ds.description}')

# Get environment
env = Environment.get(ws, config['environment_name'])
print(f'Environment {env.name} will be used.')

# Get compute cluster
cluster = ComputeTarget(workspace=ws, name=config['core']['ml_cluster_name'])
cluster_state = cluster.get_status()
print(f'ML cluster contains {cluster_state.current_node_count} compute nodes in {cluster_state.allocation_state} state')

Used dataset diabetes-data: Diabetes Disease Database
Environment environments-experiment will be used.
ML cluster contains 1 compute nodes in Steady state


Скопируем необходимые скрипты в директорию Эксперимента:

In [8]:
!cp scripts/train_model.py $experiment_dir
!cp scripts/register_model.py $experiment_dir
!ls $experiment_dir

register_model.py  train_model.py


Определение Шагов ML Конвейера:

In [9]:
## Import packages related with Azure ML Pipeline 
from azureml.pipeline.core import Pipeline
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.core.runconfig import RunConfiguration


## Step 0: Prepare
# Set Pipeline Run Configuration
pipeline_run_config = RunConfiguration()
pipeline_run_config.target = cluster
pipeline_run_config.environment = env

# Create a PipelineData (temporary Data Reference) for the model folder
model_dir = PipelineData('model_dir', datastore=ws.get_default_datastore())


## Step 1: Train model
estimator = SKLearn(source_directory=experiment_dir,
                    compute_target=cluster,
                    environment_definition=pipeline_run_config.environment,
                    entry_script='train_model.py')


train_model_step = EstimatorStep(name='Train Model Step',
                                 estimator=estimator, 
                                 estimator_entry_script_arguments=['--output_dir', model_dir],
                                 inputs=[data_ds.as_named_input('data')],
                                 outputs=[model_dir],
                                 compute_target=cluster,
                                 allow_reuse=True)

## Step 2: Register trained model
register_model_step = PythonScriptStep(name='Register Model Step',
                                       source_directory=experiment_dir,
                                       script_name='register_model.py',
                                       arguments=['--model_dir', model_dir],
                                       inputs=[model_dir],
                                       compute_target=cluster,
                                       runconfig=pipeline_run_config,
                                       allow_reuse=True)

## Complete
print('Pipeline steps were completely defined.')

WARNING - If environment_definition or conda_dependencies_file_path is specified, Azure ML will not install any framework related packages on behalf of the user.


Pipeline steps were completely defined.


Запуск созданного ML-конвейера:

In [ ]:
# Construct the pipeline
pipeline_steps = [train_model_step, register_model_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline was built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name=config['experiment_name'])

pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")

pipeline_run.wait_for_completion(show_output=True)

_Перейдети по ссылкам, которые будут появляться в `output` выполнения ML Конвейра, чтобы проследить за статусом исполнения._

## Результат

Получим список всех зарегистрированных ML моделей и найдем модель, обученную в этом Эксперименте: 

In [11]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t', tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t', prop_name, ':', prop)

diabetes-predict-model v3
	 Lab : 5A
diabetes-predict-model v2
	 Lab : 5B
	 AUC : 0.8576173518134779
	 Accuracy : 0.787
diabetes-predict-model v1
	 Lab : 4B
	 AUC : 0.8576982541474856
	 Accuracy : 0.7876666666666666


Это простой пример, призванный продемонстрировать принцип. На самом деле вы можете встроить более сложную логику в этапы конвейера - например, оценить модель по некоторым тестовым данным, чтобы вычислить метрику производительности, такую как AUC или точность, сравнить метрику с метрикой любых ранее зарегистрированных версий модели и зарегистрировать новую модель только в том случае, если она работает лучше.